In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
pd.options.display.max_columns = None

df = pd.read_csv('my_csvs/last_30_seconds_down_1.csv',low_memory=False)
df.game_id.sample().item()

400868198

In [41]:
df = df.loc[df.possession_before.notnull()]
df = df.loc[df.half < 3]
df.game_id.nunique()

3571

In [42]:
for id,sub_df in tqdm(df.groupby('game_id')):

    last_row_df = sub_df.iloc[len(sub_df)-1,:].to_frame().transpose()

    if last_row_df.home_score.item() > last_row_df.away_score.item():

        df.loc[df.game_id == id,'winner'] = sub_df.home.unique().item()

    else:

        df.loc[df.game_id == id,'winner'] = sub_df.away.unique().item()

100%|██████████| 3571/3571 [00:05<00:00, 629.24it/s]


In [10]:
df.game_id.sample(2)

102006    401089974
100145    401087532
Name: game_id, dtype: int64

In [45]:
for game_id,sub_df in df.groupby('game_id'):

    for idx_value, play in sub_df.to_dict(orient = 'index').items():

        if np.abs(play['score_diff']) !=1:
            continue
        
        game_winner = play['winner']
        home_team = play['home']
        away_team = play['away']
        
        # If the home team is up 1
        if play['score_diff'] == 1:
            # And the away team won
            if game_winner == away_team:
                # But the home team had the ball
                if play['possession_after'] == home_team:
                    df.loc[df.index == idx_value,'criteria_satisfied'] = 1
                    break
                
        # If the away team is up 1
        elif play['score_diff'] == -1:
            # But the home team won
            if game_winner == home_team:
                # And the away team had the ball
                if play['possession_after'] == away_team:

                    df.loc[df.index == idx_value,'criteria_satisfied'] = 1
                    break
        else:
            continue


In [54]:
df[df.criteria_satisfied == 1][['home','away','score_diff','possession_before','possession_after','winner','date']].query('home == "Purdue"')

,home,away,score_diff,possession_before,possession_after,winner,date
89502,Purdue,Ohio State,1,Ohio State,Purdue,Ohio State,2018-02-07
